In [8]:
import os
import shutil
import stat
import seaborn as sns
import collections
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.image as img
import random
import cv2
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as img
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from skimage.io import imread
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from keras.models import load_model

In [9]:
def gen_train_test_split(image_dir, target_dir, train_ratio=0.75):
    # Eğer hedef dizin yoksa oluştur
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    train_dir = os.path.join(target_dir, 'train')
    test_dir = os.path.join(target_dir, 'test')
    
    # Eğer eğitim ve test dizinleri yoksa oluştur
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    
    # Görüntü dosyalarını ve sınıfları yükle
    all_classes = os.listdir(image_dir)
    for class_name in all_classes:
        class_dir = os.path.join(image_dir, class_name)
        all_images = os.listdir(class_dir)
        num_images = len(all_images)
        
        # Eğitim ve test örneklerini rastgele seç
        num_train_samples = int(train_ratio * num_images)
        train_samples_selected = random.sample(all_images, num_train_samples)
        test_samples_selected = [img for img in all_images if img not in train_samples_selected]
        
        # Eğitim örneklerini kopyala
        for train_img in train_samples_selected:
            src_train = os.path.join(class_dir, train_img)
            dest_train = os.path.join(train_dir, class_name, train_img)
            if not os.path.exists(os.path.join(train_dir, class_name)):
                os.makedirs(os.path.join(train_dir, class_name))
            shutil.copy(src_train, dest_train)
        
        # Test örneklerini kopyala
        for test_img in test_samples_selected:
            src_test = os.path.join(class_dir, test_img)
            dest_test = os.path.join(test_dir, class_name, test_img)
            if not os.path.exists(os.path.join(test_dir, class_name)):
                os.makedirs(os.path.join(test_dir, class_name))
            shutil.copy(src_test, dest_test)

# Görüntü dizini
image_dir = r"C:\Users\kurt_\Desktop\food-101\images"
# Hedef dizin
target_dir = r"C:\Users\kurt_\Desktop\food-101"

# Eğitim ve test örneği oranı
train_ratio = 0.75

# Train ve test setlerini oluştur
gen_train_test_split(image_dir, target_dir, train_ratio)

In [10]:
# Method to load train-test files.
def load_train_test_data(path_to_train_imgs, path_to_test_imgs):
    X_train, y_train = load_images(path_to_train_imgs)
    X_test, y_test = load_images(path_to_test_imgs)
    return X_train, y_train, X_test, y_test

# Generate train-test files. 
if not os.path.isdir("C:/Users/kurt_/Desktop/food-101/test") and not os.path.isdir("C:/Users/kurt_/Desktop/food-101/train"):
    gen_train_test_split()  
    len_train = len(os.listdir("C:/Users/kurt_/Desktop/food-101/train"))
    len_test = len(os.listdir("C:/Users/kurt_/Desktop/food-101/test"))
    print(len_train, len_test)
else:
    print('train and test folders already exist.')
    len_train = len(os.listdir("C:/Users/kurt_/Desktop/food-101/train"))
    len_test = len(os.listdir("C:/Users/kurt_/Desktop/food-101/test"))
    print(len_train, len_test)

train and test folders already exist.
101 101


In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

# Clearing the session
tf.keras.backend.clear_session()

# Defining parameters
n_classes = 101
img_width, img_height = 224, 224
train_data_dir = r"C:\Users\kurt_\Desktop\food-101\train"
validation_data_dir = r"C:\Users\kurt_\Desktop\food-101\test"
nb_train_samples = 75750
nb_validation_samples = 25250
batch_size = 50
epochs = 30

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# MobileNetV2 model
mbv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
x = mbv2.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(n_classes, activation='softmax')(x)

model = Model(inputs=mbv2.input, outputs=predictions)
model.compile(optimizer=Adamax(), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpointer = ModelCheckpoint(filepath='best_model_101class_mobilenetv2.hdf5', save_best_only=True)
csv_logger = CSVLogger('history_101class_mobilenetv2.log') # Fixed this line

# Training the model
history_101class_mobilenetv2 = model.fit(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=[checkpointer, csv_logger])

# Saving the trained model
model.save('model_trained_101class_mobilenetv2.hdf5')
print('Training completed.')


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
Epoch 1/30
1515/1515 [==============================] - 1509s 994ms/step - loss: 2.2918 - accuracy: 0.4548 - val_loss: 3.2634 - val_accuracy: 0.3605
Epoch 2/30
1515/1515 [==============================] - 906s 598ms/step - loss: 1.4638 - accuracy: 0.6317 - val_loss: 2.8758 - val_accuracy: 0.4225
Epoch 3/30
1515/1515 [==============================] - 921s 608ms/step - loss: 1.2201 - accuracy: 0.6861 - val_loss: 2.4766 - val_accuracy: 0.4972
Epoch 4/30
1515/1515 [==============================] - 858s 566ms/step - loss: 1.0656 - accuracy: 0.7214 - val_loss: 1.7110 - val_accuracy: 0.5890
Epoch 5/30
1515/1515 [==============================] - 859s 567ms/step - loss: 0.9563 - accuracy: 0.7476 - val_loss: 1.9736 - val_accuracy: 0.5511
Epoch 6/30
1515/1515 [==============================] - 854s 564ms/step - loss: 0.8743 - accuracy: 0.7668 - val_loss: 1.6058 - val_accuracy: 0.6267
Epoch 7/30
1515/1515 

In [18]:
model_path = r"C:\Users\kurt_\best_model_101class_mobilenetv2.hdf5"
test_data_path = r"C:\Users\kurt_\Desktop\food-101\test"

In [19]:
# Performans metriklerini hesaplamak için fonksiyon tanımla
def calculate_metrics(model, test_gen):
    # Modelden tahminler yap
    predictions = model.predict(test_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Gerçek sınıfları al
    true_classes = test_gen.classes
    
    # Sınıf etiketlerini al
    class_labels = list(test_gen.class_indices.keys())

    # Doğruluk, hassasiyet, geri çağırma ve F1 puanını hesapla
    accuracy = accuracy_score(true_classes, predicted_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')

    return accuracy, precision, recall, f1

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from keras.models import load_model
import numpy as np


# Boyutları tanımla
height, width = 224, 224

# Batch boyutunu tanımla
batch_size = 16

# Test veri üreteci oluştur
test_data_gen = ImageDataGenerator(rescale=1. / 255)
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical', shuffle=False)

# Modeli yükle
model = load_model(model_path, compile=False)

# Performans metriklerini hesapla
accuracy, precision, recall, f1_score = calculate_metrics(model, test_gen)

# Hesaplanan metrikleri yazdır
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Found 25250 images belonging to 101 classes.
1579/1579 [==============================] - 85s 53ms/step
Accuracy: 0.696
Precision: 0.7285602710387407
Recall: 0.696
F1 Score: 0.6987808495076148
